<a href="https://colab.research.google.com/github/HayateSato/Pra/blob/main/9%E7%AB%A0_%E5%B0%8F%E8%A6%8F%E6%A8%A1%E6%A9%9F%E6%A2%B0%E5%AD%A6%E7%BF%92%E3%82%B7%E3%82%B9%E3%83%86%E3%83%A0%E3%82%92%E4%BD%9C%E6%88%90%E3%81%99%E3%82%8B%EF%BC%91%EF%BC%90%E6%9C%AC%E3%83%8E%E3%83%83%E3%82%AF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 下記セルを実行すると、Googleドライブへの接続を求められます。
# Googleアカウントにログインして進めてください。
import os
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
working_dir = 'MLSys_100Knocks' #　※※自分で作成したフォルダパスが異なる場合こちらを変更してください。※※
path = f'/content/drive/MyDrive/{working_dir}/本章/9章'
os.chdir(path)

# ９章_小規模機械学習システムを作成する１０本ノック



### ノック８１：フォルダ生成をして初期の変数定義をしよう

##### ＊＊必ずデータやモデルの配置をおこなってください。＊＊

### ノック８２：更新データを読み込んで店舗別データを作成しよう

### ノック８３：月次店舗データの更新をしよう

### ノック８４：機械学習用データの作成と更新をしよう

### ノック８５：機械学習モデル用の事前データ加工をしよう

### ノック８６：機械学習モデルの構築・評価をしよう

### ノック８７：新規データ予測に向けた下準備をしよう

### ノック８８：新規データの予測をしよう

### ノック８９：現場向けレポートを作成し出力しよう

#### ＊＊次ノックに行く前に、ノック81のtg_ymを、202005、202006、202007、202008の順番に指定し、実行してください。＊＊

### ノック９０：機械学習モデルの精度推移を可視化しよう